# Importar librerías

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
pd.set_option('display.float_format', '{:.6f}'.format)

## Verificar estructura de la columnas de "genres"

In [73]:
df = pd.read_csv('movies_metadata.csv')
df['genres'][0]

c:\users\hansr\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [74]:
df['genres'] = df['genres'].str.replace("\'","\"")

In [75]:
df['genres'][0]

'[{"id": 16, "name": "Animation"}, {"id": 35, "name": "Comedy"}, {"id": 10751, "name": "Family"}]'

## Separar el json genres en columnas

In [76]:
import json
json_cols = ['genres']

def new_json(x):
    return json.loads(x)

for x in json_cols:
    df[x] = df[x].apply(new_json)

In [77]:
def new_json2(x):

    ls = []

    for y in range(len(x[0])):

        for k, v in x[0][y].items():
            ls.append(str(k)+ "_" +str(v))
   
    for z in range(len(ls)):

        if ls[z] not in df.columns:
            df[ls[z]] = 0
            df[ls[z]].iloc[x.name] = 1
        else:
            df[ls[z]].iloc[x.name] = 1
    return

for x in json_cols:
    df[[x]].apply(new_json2, axis=1)

In [78]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'id_16', 'name_Animation', 'id_35',
       'name_Comedy', 'id_10751', 'name_Family', 'id_12', 'name_Adventure',
       'id_14', 'name_Fantasy', 'id_10749', 'name_Romance', 'id_18',
       'name_Drama', 'id_28', 'name_Action', 'id_80', 'name_Crime', 'id_53',
       'name_Thriller', 'id_27', 'name_Horror', 'id_36', 'name_History',
       'id_878', 'name_Science Fiction', 'id_9648', 'name_Mystery', 'id_10752',
       'name_War', 'id_10769', 'name_Foreign', 'id_10402', 'name_Music',
       'id_99', 'name_Documentary', 'id_37', 'name_Western', 'id_10770',
       'name_TV Movie', 'name_Carousel Productions', 'id_11176'

## Hacer merge los df's

In [79]:
linksdf = pd.read_csv('links_small.csv')
linksdf.head()

,movieId,imdbId,tmdbId
0,1,114709,862.000000
1,2,113497,8844.000000
2,3,113228,15602.000000
3,4,114885,31357.000000
4,5,113041,11862.000000


In [80]:
linksdf.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [81]:
linksdf['tmdbId'] = linksdf['tmdbId'].astype(str)
linksdf['tmdbId'] = linksdf['tmdbId'].str.replace("\.0","")

In [82]:
linksdf['tmdbId'] = linksdf['tmdbId'].astype(object)

In [83]:
new_df = pd.merge(df,linksdf, left_on = 'id', right_on = 'tmdbId')

In [91]:
ratingsdf = pd.read_csv('ratings_small.csv')
ratingsfiltered = ratingsdf.drop('timestamp', axis = 1)

In [92]:
finaldf = pd.merge(new_df, ratingsfiltered, on = 'movieId')

In [96]:
df_x = finaldf.filter(regex=("^name"))

In [97]:
df_x.shape

(99850, 32)

In [98]:
yfiltered = finaldf[['userId', 'movieId', 'rating']]

In [100]:
nice = yfiltered.set_index(['movieId', 'userId', 'rating'], append=True)

In [112]:
ypivot = yfiltered.pivot_table(index='movieId',columns='userId',values='rating', aggfunc=np.mean)

In [113]:
ypivot.shape

(9025, 671)

In [121]:
df_x2 = pd.concat([finaldf['movieId'], df_x], axis = 1)
xdata = df_x2.pivot_table(index='movieId',aggfunc='mean')

In [59]:
xnp = df_x.to_numpy()
ynp = rating_final.to_numpy()

In [65]:
ynp.shape

(9025, 433)

In [20]:
def cost(Y, X, T):
    return(((X @ T.T - Y) ** 2) * ((Y != 0) * 1)).sum()


def gradient(Y, X, T):
    R = (Y != 0) * 1
    hip_error = (X @ T.T - Y) * R

    return (
        hip_error @ T,
        hip_error.T @ X,
    )


def adam(
        Y,
        Xo,
        To,
        fun,
        jac,
        alpha=0.001,
        beta1=0.9,
        beta2=0.999,
        epsilon=0.0000001,
        max_iter=1000
):
    xm = cp.zeros(Xo.shape)
    tm = cp.zeros(To.shape)

    xv = cp.zeros(Xo.shape)
    tv = cp.zeros(To.shape)

    X, T, t = Xo, To, 0

    while t < max_iter:
        t += 1

        xg, tg = jac(Y, X, T)

        print(f'{t} \t loss={fun(Y, X, T).item():,.2f}')

        xm = beta1 * xm + (1 - beta1) * xg
        tm = beta1 * tm + (1 - beta1) * tg

        xv = beta2 * xv + (1 - beta2) * xg * xg
        tv = beta2 * tv + (1 - beta2) * tg * tg

        xmh = xm / (1 - beta1 ** t)
        tmh = tm / (1 - beta1 ** t)

        xvh = xv / (1 - beta2 ** t)
        tvh = tv / (1 - beta2 ** t)

        X -= alpha * xmh / (cp.sqrt(xvh) + epsilon)
        T -= alpha * tmh / (cp.sqrt(tvh) + epsilon)

    return X, T


In [23]:
xnp = df_x.to_numpy()
ynp = rating_final.to_numpy()

In [27]:
ynp.shape

(9066, 671)

In [28]:
xnp.shape

(9042, 32)

In [ ]:
xtrain, xvalidate, xtest = np.split(df_x, [int(len(df_x)*0.7), int(len(df_x)*0.85)])
ytrain, yvalidate, ytest = np.split(arr, [int(len(arr)*0.7), int(len(arr)*0.85)])

In [21]:
'''
valores = finaldf['movieId'].to_frame()
rdf = ratingsdf2.to_frame()
rdf2 = rdf.sort_values(by = 'movieId')
for i in [valores, rdf2]:
    i.sort_index(axis = 1 ,inplace=True)
'''

"\nvalores = finaldf['movieId'].to_frame()\nrdf = ratingsdf2.to_frame()\nrdf2 = rdf.sort_values(by = 'movieId')\nfor i in [valores, rdf2]:\n    i.sort_index(axis = 1 ,inplace=True)\n"